In [1]:
import pandas as pd
import sqlite3
import yfinance as yf
import time

In [2]:
#Need a list of stocks in the S&P 500

#Let's get data from wikipedia
#We only need the first table, which lists the 505 stocks in the index and includes their symbols
wiki_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

wiki_df[["Symbol","Security","CIK"]]

#List of dates to skip over (12/5/18 for the state funeral of George H.W. Bush)
SkipDates = ['2018-12-05','2022-02-17','2022-02-18','2022-02-19','2022-02-20','2022-02-21',
             '2022-02-22','2022-02-23','2022-02-24','2022-02-25','2022-02-26','2022-02-27','2022-02-28']


In [3]:
#connect to the S&P 500 database file and create an associated cursor
conn = sqlite3.connect("SP500_stats_5y.db")
c = conn.cursor()


c.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='CompanyInfo'")
if len(c.fetchall()) == 0:
    #Using this if statement to make sure this only runs if the table doesn't exist already
    #create a table to hold company information for the S&P 500 stocks
    c.execute("""CREATE TABLE CompanyInfo(
                        CIK int,
                        Symbol text,
                        Name text,
                        Sector text,
                        Industry text,
                        PRIMARY KEY (Symbol)
                    )""")
    #commit this table to the database
    conn.commit()

In [4]:
def GetStockInfo(ind):
    
    SymbolStr = wiki_df['Symbol'][ind]
    CIK = wiki_df['CIK'][ind]
    
    tickval = yf.Ticker(SymbolStr.replace('.','-'))
    tickval.info
    
    
    if 'longName' in tickval.info:
        name = tickval.info['longName'].replace("'","").replace('^','')
    else:
        name = wiki_df['Security'][ind]
    if 'sector' in tickval.info:
        sector = tickval.info['sector'].replace("'","").replace('^','')
    else:
        sector = 'N/A'
    if 'industry' in tickval.info:
        industry = tickval.info['industry'].replace("'","").replace('^','')
    else:
        industry = 'N/A'
    
    info_str = str(CIK)+",'"+SymbolStr.replace('^','')+"','"+name+"','"+sector+"','"+industry+"'"
    c.execute("INSERT INTO CompanyInfo VALUES ("+info_str+")")
    conn.commit()
    
    #Now get archival trading info
    hist = tickval.history(period="61mo")

    
    print(str(i)+","+info_str+","+str(len(hist)-1))
    
    c.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='"+SymbolStr.replace('.','_').replace('^','')+"_stats'")
    if len(c.fetchall()) == 0:
        #Using this if statement to make sure this only runs if the table doesn't exist already
        #create a table for daily stock statistics
        c.execute("""CREATE TABLE """+SymbolStr.replace('.','_').replace('^','')+"""_stats(
                            date_int int,
                            Date text,
                            Open float,
                            High float,
                            Low float,
                            Close float,
                            PRIMARY KEY (date_int)
                        )""")
        #commit this table to the database
        conn.commit()
    
    #just go to len(hist)-1 since the last entry is prone to nan values
    for j in range(len(hist)-1):
        Date = str(hist.index[j]).split()[0]
        if not Date in SkipDates:
            date_int = str(hist.index[j] - hist.index[0]).split()[0]
            Open = hist['Open'][j]
            High = hist['High'][j]
            Low = hist['Low'][j]
            Close = hist['Close'][j]

            stat_str = date_int+",'"+Date+"',"+str(Open)+","+str(High)+","+str(Low)+","+str(Close)
            #print(stat_str)

            c.execute("INSERT INTO "+SymbolStr.replace('.','_').replace('^','')+"_stats VALUES ("+stat_str+")")
            conn.commit()
    

In [5]:
c.execute("SELECT * FROM CompanyInfo")
companies_df = pd.DataFrame(c.fetchall(), columns = [x[0] for x in c.description])

companies_df

,CIK,Symbol,Name,Sector,Industry
0,66740,MMM,3M Company,Industrials,Conglomerates
1,91142,AOS,A. O. Smith Corporation,Industrials,Specialty Industrial Machinery
2,1800,ABT,Abbott Laboratories,Healthcare,Medical Devices
3,1551152,ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers—General
4,815094,ABMD,"Abiomed, Inc.",Healthcare,Medical Devices
...,...,...,...,...,...
502,1682852,MRNA,"Moderna, Inc.",Healthcare,Biotechnology
503,1821825,OGN,Organon & Co.,Healthcare,Drug Manufacturers—General
504,1781335,OTIS,Otis Worldwide Corporation,Industrials,Specialty Industrial Machinery
505,813828,VIAC,Paramount Global,Communication Services,Entertainment


In [6]:
Issues = []
for i in range(len(wiki_df)):
    if wiki_df['Symbol'][i] in list(companies_df['Symbol']):
        print("Already have "+wiki_df['Symbol'][i]+".")
    else:
        try:
            GetStockInfo(i)
        except Exception as e:
            Issues.append(wiki_df['Symbol'][i])
            print("Hmmm. Had some trouble with "+wiki_df['Symbol'][i]+".")
            print(e)


Already have MMM.
Already have AOS.
Already have ABT.
Already have ABBV.
Already have ABMD.
Already have ACN.
Already have ATVI.
Already have ADM.
Already have ADBE.
Already have ADP.
Already have AAP.
Already have AES.
Already have AFL.
Already have A.
Already have AIG.
Already have APD.
Already have AKAM.
Already have ALK.
Already have ALB.
Already have ARE.
Already have ALGN.
Already have ALLE.
Already have LNT.
Already have ALL.
Already have GOOGL.
Already have GOOG.
Already have MO.
Already have AMZN.
Already have AMCR.
Already have AMD.
Already have AEE.
Already have AAL.
Already have AEP.
Already have AXP.
Already have AMT.
Already have AWK.
Already have AMP.
Already have ABC.
Already have AME.
Already have AMGN.
Already have APH.
Already have ADI.
Already have ANSS.
Already have ANTM.
Already have AON.
Already have APA.
Already have AAPL.
Already have AMAT.
Already have APTV.
Already have ANET.
Already have AIZ.
Already have T.
Already have ATO.
Already have ADSK.
Already have 

In [7]:
c.execute("SELECT * FROM CompanyInfo")
companies_df = pd.DataFrame(c.fetchall(), columns = [x[0] for x in c.description])

companies_df

,CIK,Symbol,Name,Sector,Industry
0,66740,MMM,3M Company,Industrials,Conglomerates
1,91142,AOS,A. O. Smith Corporation,Industrials,Specialty Industrial Machinery
2,1800,ABT,Abbott Laboratories,Healthcare,Medical Devices
3,1551152,ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers—General
4,815094,ABMD,"Abiomed, Inc.",Healthcare,Medical Devices
...,...,...,...,...,...
502,1682852,MRNA,"Moderna, Inc.",Healthcare,Biotechnology
503,1821825,OGN,Organon & Co.,Healthcare,Drug Manufacturers—General
504,1781335,OTIS,Otis Worldwide Corporation,Industrials,Specialty Industrial Machinery
505,813828,VIAC,Paramount Global,Communication Services,Entertainment


In [8]:
#Clean out any troublesome entries so the script can be re-run if needed to finish filling in entries
for BadSymbol in Issues:
    print("Cleaning out entries for "+BadSymbol)
    c.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='"+BadSymbol.replace('.','_')+"_stats'")
    if len(c.fetchall()) > 0:
        c.execute("DROP TABLE "+BadSymbol.replace('.','_')+"_stats;")
        conn.commit()
    c.execute("DELETE FROM CompanyInfo WHERE Symbol='"+BadSymbol.replace('.','_')+"';")
    conn.commit()